In [3]:
import pysam
from tqdm import tqdm

dir="/Users/mhryansohn/Desktop/01.Workspace/01.Projects/03.AK1-PacBio/01.DNA/Public_Data/HG002"
hg002 = pysam.Ali
ak1 = pysam.AlignmentFile("chr20_500000-1000000_AK1.bam", 'rb')
a = next(ak1)
b = next(ak1)

a.get_tag('MM')
a.get_tag('ML')
a.query_name

#Negative 일 경우
for pos, prob in list(a.modified_bases.values())[0]:
    if a.seq[pos-1: pos+1] != 'CG':
        print('fuck')
#Positive 일 경우
for pos, prob in list(b.modified_bases.values())[0]:
    if b.seq[pos: pos+2] != 'CG':
        print('fuck')
        
        
        


AttributeError: module 'pysam' has no attribute 'Ali'

modkit motif-bed /mnt/mone/Project/AK1_PacBio/Tools/Reference/GRCh38/DNA/hg38.analysisSet.fa CG 0 > hg38.analysisSet_CG.bed
modkit extract chr20_500000-1000000_AK1.bam chr20_500000-1000000_AK1_CpG.tsv --region chr20 --ref /mnt/mone/Project/AK1_PacBio/Tools/Reference/GRCh38/DNA/hg38.analysisSet.fa --include-bed hg38.analysisSet_CG.bed